# fmu.sumo.explorer

Package for browsing cases and surfaces in Sumo.

In [ ]:
from fmu.sumo.explorer import Explorer
# import xtgeo
from io import BytesIO
%matplotlib inline

## Class: Explorer
Search for cases in SUMO

Methods:
- get_fields()
- get_users()
- get_status()
- get_cases()

In [ ]:
sumo = Explorer(env="prod")

### Explorer.get_fields()

Returns:

```
Dict
{
    FIELD_NAME: CASE_COUNT
}
```

In [ ]:
fields = sumo.get_fields()

fields

### Explorer.get_users()

Returns:

```
Dict
{
    USER_NAME: CASE_COUNT
}
```

In [ ]:
users = sumo.get_users()

users

### Explorer.get_status()

Returns:

`Dict`

```
{
    STATUS: CASE_COUNT
}
```

In [ ]:
status = sumo.get_status()

status

### Explorer.get_cases()

Params:
- status: List[string]
- fields: List[string]
- users: List[string]

Returns: `DocumentCollection`

In [ ]:
my_cases = sumo.get_cases(
    status=["keep", "official"],
    fields=["JOHAN SVERDRUP"],
    #users=["peesv", "jsorb"]
)

print(len(my_cases))

for case in my_cases:
    print(case.case_name)

### Explorer.get_case_from_name()

Params:
- name: string

Returns: `Case`

In [ ]:
my_case = sumo.get_case_by_name("drogon_design_2022_11-01")

print(f"{my_case.name}: {my_case.sumo_id}")

## Class: Case

Represents a case in Sumo and has methods for finding surfaces belonging to the case.

Some of the case meta data is accessible as attributes, the rest can be extracted from `Case.meta_data`.

Attributes:
- meta_data: dict
- sumo_id: uuid
- fmu_id: uuid
- case_name: string
- field_name: string
- status: string
- user: string

Methods:
- get_object_types()
- get_iterations()
- get_realizations()
- get_object_names()
- get_object_tag_names()
- get_object_aggregations()
- get_object_timespans()
- get_object()

Raw metadata object can be accessed at `Case.meta_data`

In [ ]:
# Some case info as attributes
print(my_case.sumo_id)
print(my_case.fmu_id)
print(my_case.name)
print(my_case.field_name)
print(my_case.status)
print(my_case.user)

### Case.get_object_types()

Returns:
```
Dict
{
    OBJECT_NAME: OBJECT_COUNT
}
```

In [ ]:
object_types = my_case.get_object_types()

object_types

### sumo.get_dict_of_case_names()

Returns:

```
Dict
{
    CASE_NAME: SUMO_ID
}
```

In [ ]:
sumo.get_dict_of_case_names()

### sumo.get_summary_object_ids()

Returns:

```
Dict
{
    VECTOR_NAME: OBJECT_ID
}
```

In [ ]:
my_case.get_summary_object_ids(size=1000)

### sumo.get_object_ids()

Returns:

```
Dict
{
    REAL_NR: OBJECT_ID
}
```

In [ ]:
my_case.get_object_ids("VOLANTIS GP. Base", "FACIES_Fraction_Offshore")

### sumo.get_aggregated_object_ids()

Returns:

```
Dict
{
    OBJECT_NAME: {AGGREGATION_TYPE: OBJECT_ID}
}
```

In [ ]:
# Not added yet

### Case.get_iterations()

Returns:
```
Dict
{
    ITERATION_ID: OBJECT_COUNT
}
```

In [ ]:
iterations = my_case.get_iterations()

iterations

### Case.get_realizations()

Params:
- iteration_id: number

Returns:
```
Dict
{
    REALIZATION_ID: OBJECT_COUNT
}
```

In [ ]:
realizations = my_case.get_realizations(iteration_id=0)

realizations[0]

#### Example: filter down to unique surface object

This example uses the `get_object_property_values` in several steps to get values that uniquley identifies a surface object.

In [ ]:
# Get a surface object from specific realization
surfaces = my_case.get_objects(
    object_type="surface",
    object_names=["Volon Fm. Top"],
    # tag_names=["amplitude_full_max"],
    # time_intervals=["2019-10-01 - 2020-10-01"],
    iteration_ids=[0],
    realization_ids=[0]
)

s = surfaces[0]

print(s.name)
print(s.tag_name)

In [ ]:
# Get all versions of surface object (1 from each realization) for a specified iteration
surfaces = my_case.get_objects(
    object_type="surface",
    object_names=["Volon Fm. Top"],
    # tag_names=["DS_extract_geogrid"],
    iteration_ids=[0]
)

surf = surfaces[0]
print(surf.realization_id)
print(surf.iteration_id)
print(surf.name)